In [3]:
import pandas as pd
import ast

In [ ]:
master_full_data = []
with pd.read_csv("/gpfs/milgram/project/rtaylor/shared/master_data/master_full.csv",chunksize=30000) as reader:
    for idx, chunk in enumerate(reader):
        print(idx)
        master_full_data.append(chunk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [ ]:
master_full_data = pd.concat(master_full_data)

In [ ]:
master_full_data

In [ ]:
full_pt_dispos = master_full_data[["PAT_ENC_CSN_ID", 'disposition']]


In [ ]:
del master_full_data
import gc
gc.collect()

In [ ]:
full_pt_dispos

In [ ]:
annotations1 = pd.read_csv("~/project/UTI_Noise_data/500_random_notes_exclusions.csv")
annotations2 = pd.read_csv("~/project/UTI_Noise_data/1000_random_notes_exclusions_comb.csv")

In [ ]:
data = pd.read_csv("/home/vs428/project/UTI_Noise_data/provider_notes_full.tsv", sep="\t")

In [ ]:
data['PAT_ENC_CSN_ID'].nunique(), data.shape

In [ ]:
diags = pd.read_csv("/home/vs428/project/Data/ed_diagnoses.csv")

In [ ]:
data_diags = diags[diags['PAT_ENC_CSN_ID'].isin(data['PAT_ENC_CSN_ID'])]

In [ ]:
diags.shape, data_diags.shape

In [ ]:
# Acute pyelonephritis, unspecified Urinary tract infection, Diphtheritic cystitis, Salmonella pyelonephritis
# UTI following delivery, unspecified, UTI of urinary tract in pregnancy, unspecified trimester, UTI following an ectopic and molar pregnancy, UTI following incomplete spontaneous abortion
UTI_DIAG_CODES = ['N10', "N39.0", "A36.85", "A02.25", "O86.20", "O23.40", "O08.83", "O03.38"]
# All cystitis codes
UTI_DIAG_UPPERCATS = ['N30']
interstitial_cystitis_CODE = "N30.1"

In [ ]:
diags['Value_Cat'] = diags['Value'].str.slice(0,3)


In [ ]:
uti_diags = diags[(diags['Value'].isin(UTI_DIAG_CODES) | 
                        diags['Value_Cat'].isin(UTI_DIAG_UPPERCATS))
           & ~(diags['Value'] == interstitial_cystitis_CODE)]

# they didn't have anything in the differential, but they did an antbiotic and 
# cystitits, pylenorphritis, general UTI
# the culture shows up as more than 10K CFUS (lines of 273-286), 10K >= 


In [ ]:
# Get the over 18 encounter IDs
with open("/home/vs428/project/UTI_Noise_data/Over_18.txt") as f:
    IDs_18_str = f.read()

In [ ]:
IDs_18  = ast.literal_eval(IDs_18_str)

In [ ]:
data_over_18 = data[data['PAT_ENC_CSN_ID'].isin(IDs_18)]

In [ ]:
data.shape, data_over_18.shape

# Using the people over 18, split into 4 25% groups

In [ ]:
pts_with_uti_diags = data_over_18[data_over_18['PAT_ENC_CSN_ID'].isin(uti_diags["PAT_ENC_CSN_ID"])]
pts_wout_uti_diags = data_over_18[~data_over_18['PAT_ENC_CSN_ID'].isin(uti_diags["PAT_ENC_CSN_ID"])]
print(f"Patients with UTI Diags: {pts_with_uti_diags['PAT_ENC_CSN_ID'].nunique()} \nPatients without UTI Diags: {pts_wout_uti_diags['PAT_ENC_CSN_ID'].nunique()}")

In [ ]:
admitted_pts = full_pt_dispos[full_pt_dispos['disposition'] == "Admit"]
discharged_pts = full_pt_dispos[full_pt_dispos['disposition'] == "Discharge"]

In [ ]:
admit_uti_diags = pts_with_uti_diags[pts_with_uti_diags["PAT_ENC_CSN_ID"].isin(admitted_pts["PAT_ENC_CSN_ID"])]
admit_no_utis = pts_wout_uti_diags[pts_wout_uti_diags["PAT_ENC_CSN_ID"].isin(admitted_pts["PAT_ENC_CSN_ID"])]

discharge_uti_diags = pts_with_uti_diags[pts_with_uti_diags["PAT_ENC_CSN_ID"].isin(discharged_pts["PAT_ENC_CSN_ID"])]
discharge_no_utis = pts_wout_uti_diags[pts_wout_uti_diags["PAT_ENC_CSN_ID"].isin(discharged_pts["PAT_ENC_CSN_ID"])]


In [ ]:
npts = data_over_18['PAT_ENC_CSN_ID'].nunique()
x = (f"Admitted patients with UTI Diags: {admit_uti_diags['PAT_ENC_CSN_ID'].nunique()} ({admit_uti_diags['PAT_ENC_CSN_ID'].nunique()/npts:.3f})\n"
     f"Admitted patients without UTI Diags: {admit_no_utis['PAT_ENC_CSN_ID'].nunique()} ({admit_no_utis['PAT_ENC_CSN_ID'].nunique()/npts:.3f})")
print(x)
y = (f"Discharged patients with UTI Diags: {discharge_uti_diags['PAT_ENC_CSN_ID'].nunique()} ({discharge_uti_diags['PAT_ENC_CSN_ID'].nunique()/npts:.3f})\n"
     f"Discharged patients without UTI Diags: {discharge_no_utis['PAT_ENC_CSN_ID'].nunique()} ({discharge_no_utis['PAT_ENC_CSN_ID'].nunique()/npts:.3f})")
print(y)
print(f"Total patients: {data_over_18['PAT_ENC_CSN_ID'].nunique()}")


In [ ]:
31652 + 221367 + 35031 + 291364

In [ ]:
admits_in_data = data_over_18[data_over_18['PAT_ENC_CSN_ID'].isin(admitted_pts["PAT_ENC_CSN_ID"])]
admits_in_data['PAT_ENC_CSN_ID'].nunique() / data_over_18['PAT_ENC_CSN_ID'].nunique()

# Create New dataset after dropping any from old IDs

In [ ]:
prev_annotated_enc_ids = list(set(annotations1['PAT_ENC_CSN_ID'].unique().tolist()).union(set(annotations2['PAT_ENC_CSN_ID'].unique().tolist())))

In [ ]:
admit_uti_diags_new = admit_uti_diags[~admit_uti_diags['PAT_ENC_CSN_ID'].isin(prev_annotated_enc_ids)]
admit_no_utis_new = admit_no_utis[~admit_no_utis['PAT_ENC_CSN_ID'].isin(prev_annotated_enc_ids)]

discharge_uti_diags_new = discharge_uti_diags[~discharge_uti_diags['PAT_ENC_CSN_ID'].isin(prev_annotated_enc_ids)]
discharge_no_utis_new = discharge_no_utis[~discharge_no_utis['PAT_ENC_CSN_ID'].isin(prev_annotated_enc_ids)]


In [ ]:
npts = data_over_18['PAT_ENC_CSN_ID'].nunique()
x = (f"Admitted patients with UTI Diags: {admit_uti_diags_new['PAT_ENC_CSN_ID'].nunique()} ({admit_uti_diags_new['PAT_ENC_CSN_ID'].nunique()/npts:.3f})\n"
     f"Admitted patients without UTI Diags: {admit_no_utis_new['PAT_ENC_CSN_ID'].nunique()} ({admit_no_utis_new['PAT_ENC_CSN_ID'].nunique()/npts:.3f})")
print(x)
y = (f"Discharged patients with UTI Diags: {discharge_uti_diags_new['PAT_ENC_CSN_ID'].nunique()} ({discharge_uti_diags_new['PAT_ENC_CSN_ID'].nunique()/npts:.3f})\n"
     f"Discharged patients without UTI Diags: {discharge_no_utis_new['PAT_ENC_CSN_ID'].nunique()} ({discharge_no_utis_new['PAT_ENC_CSN_ID'].nunique()/npts:.3f})")
print(y)
print(f"Total patients: {data_over_18['PAT_ENC_CSN_ID'].nunique()}")

# There are slight changes in all categories above, so we did get the right set before, but now it's much more principled

In [98]:
annotationv2_dataset = pd.concat([admit_uti_diags_new.sample(375), admit_no_utis_new.sample(375), discharge_uti_diags_new.sample(375), discharge_no_utis_new.sample(375)])


In [103]:
# Randomize the order!!!
annotationv2_dataset = annotationv2_dataset.sample(frac=1)

In [116]:
annotationv2_dataset.to_csv("~/project/UTI_Noise_data/1500_25per_each_v1.csv")

# Create Dataset for Resident Annotation

In [68]:
import pandas as pd
import jsonlines
import ast
import json
import random

In [22]:

gold_anns = []
with open('/home/vs428/project/UTI_Noise_data/UTI_symptoms_GOLD_v1.jsonl') as f:
    for line in f:
        gold_anns.append(json.loads(line))

In [23]:
print(len(gold_anns))

60


In [24]:
iscoe_anns = []
with jsonlines.open("/home/vs428/project/UTI_Noise_data/UTI_symptoms_CORRECT-iscoe.jsonl") as reader:
    for line in reader:
        iscoe_anns.append(line)
        

taylor_anns = []
with jsonlines.open("/home/vs428/project/UTI_Noise_data/UTI_symptoms_CORRECT-rat4.jsonl") as reader:
    for line in reader:
        taylor_anns.append(line)
        
        
# gold_anns = []
# with open('/home/vs428/project/UTI_Noise_data/UTI_symptoms_CORRECT-rat4.jsonl') as f:
#     for line in f:
#         gold_anns.append(json.loads(line))
        

# gold_anns = []
# with open('/home/vs428/project/UTI_Noise_data/UTI_symptoms_GOLD_v1.jsonl') as f:
#     for line in f:
#         gold_anns.append(json.loads(line))        

In [25]:
taylor_input_hashes = set([taylor_ann['_input_hash'] for taylor_ann in taylor_anns])


In [26]:
def compare_two_lists(list1: list, list2: list) -> bool:
    """
    Compare two lists and logs the difference.
    :param list1: first list.
    :param list2: second list.
    :return:      if there is difference between both lists.
    """
    diff = [i for i in list1 + list2 if i not in list1 or i not in list2]
    result = len(diff) == 0
    if not result:
        pass
        # print(f'There are {len(diff)} differences:\n{diff[:5]}')
    return result

In [38]:
len(taylor_input_hashes.intersection(set([iscoe_ann['_input_hash'] for iscoe_ann in iscoe_anns])))

85

In [41]:
counter = 0
noncounter = 0
agreed_anns = []

for iscoe_ann in iscoe_anns:
    if iscoe_ann['_input_hash'] in taylor_input_hashes:
        cur_taylor_ann = [taylor_ann['spans'] for taylor_ann in taylor_anns if taylor_ann['_input_hash'] == iscoe_ann['_input_hash']][0]
        # print(iscoe_ann['spans'])
        # print(cur_taylor_ann)
        
        if not compare_two_lists(iscoe_ann['spans'], cur_taylor_ann):
            counter += 1
            print(iscoe_ann['spans'])
            print(cur_taylor_ann)
            agreed_anns.append(iscoe_ann)
            # break
        else:
            noncounter += 1
            # print(iscoe_ann['spans'])
            # print(cur_taylor_ann)

    # break


[{'start': 235, 'end': 248, 'text': 'low back pain', 'source': '/home/kl533/UTI_symptoms/models/uti_symptoms_nontransformer/model-best/', 'input_hash': 708283577, 'token_start': 44, 'token_end': 46, 'label': 'Low_back_pain'}, {'start': 250, 'end': 267, 'token_start': 48, 'token_end': 49, 'label': 'Dysuria'}, {'start': 379, 'end': 396, 'text': 'urinary frequency', 'source': '/home/kl533/UTI_symptoms/models/uti_symptoms_nontransformer/model-best/', 'input_hash': 708283577, 'token_start': 72, 'token_end': 73, 'label': 'Urinary_frequency'}, {'start': 402, 'end': 409, 'token_start': 75, 'token_end': 75, 'label': 'Dysuria'}, {'start': 429, 'end': 459, 'token_start': 81, 'token_end': 86, 'label': 'Low_back_pain'}]
[{'start': 205, 'end': 223, 'token_start': 40, 'token_end': 41, 'label': 'Urinary_frequency'}, {'start': 235, 'end': 248, 'text': 'low back pain', 'source': '/home/kl533/UTI_symptoms/models/uti_symptoms_nontransformer/model-best/', 'input_hash': 708283577, 'token_start': 44, 'token_

In [95]:
all_accept_ann_hashes = set([agreed_ann['_input_hash'] for agreed_ann in agreed_anns]).union(set([gold_ann['_input_hash'] for gold_ann in gold_anns]))


In [57]:
all_accept_ann_hashes = agreed_anns + gold_anns

In [59]:
all_accept_ann_hashes_unq = {ann['_input_hash']:ann for ann in all_accept_ann_hashes}

In [65]:
# let's us get one set of annotations for each by taking adv of the unique keying in dicts
# because we don't really care which one we keep
all_accept_ann_hashes_unq = list(all_accept_ann_hashes_unq.values())

In [75]:
random.Random(4).shuffle(all_accept_ann_hashes_unq, )

In [98]:
RESIDENT_TRAIN_SIZE = 50
resident_anns = all_accept_ann_hashes_unq[:RESIDENT_TRAIN_SIZE]

In [78]:
with jsonlines.open("/home/vs428/project/UTI_Noise_data/UTI_symptoms_GOLD_Adj_resident_train.jsonl", "w") as writer:
    writer.write_all(resident_anns)

In [86]:
# resident_anns[0].keys()

dict_keys(['text', '_input_hash', '_task_hash', 'tokens', 'spans', '_is_binary', '_view_id', 'answer', '_timestamp', '_annotator_id', '_session_id', 'sessions', 'versions', 'view_id'])

In [90]:
limited_anns = []
for ann in resident_anns:
    for span in ann['spans']:
        if "source" in span.keys():
            span.pop("source")
            span.pop("token_start")
            span.pop("token_end")    
    limited_anns.append((ann["_input_hash"], ann['text'], ann['spans']))


In [91]:
out = pd.DataFrame.from_records(limited_anns)


In [94]:
out = out.rename({0:"input_hash", 1:"TEXT", 2:"SPANS"}, axis=1)

In [99]:
assert len([x for x in out['input_hash'].tolist() if x in all_accept_ann_hashes]) == RESIDENT_TRAIN_SIZE

In [93]:
out.to_csv("/home/vs428/project/UTI_Noise_data/UTI_symptoms_GOLD_Adj_resident_train_spans.csv", index=False)



In [101]:
# prev_prodigy_input = pd.read_csv("~/project/UTI_Noise_data/1500_25per_each_v1.csv")

In [131]:
out['TEXT'].rename('text').to_csv("/home/vs428/project/UTI_Noise_data/UTI_symptoms_GOLD_Adj_resident_train_prodigy_input.csv", index=False)